<a href="https://colab.research.google.com/github/SMunira38/RiCe-vGG16-NIZHUM-/blob/main/Rice_diseases_vgg16_nizhum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Data.zip to Data.zip


In [3]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/Data.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
len(os.listdir('/tmp/Data/Train'))

3

In [4]:
!pip install torch torchvision
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.models import vgg16_bn


In [10]:
train_path = "/tmp/Data/Train/"
test_path = "/tmp/Data/Test/"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize with ImageNet mean and standard deviation
])

train_dataset = torchvision.datasets.ImageFolder(train_path, transform=transform)
test_dataset = torchvision.datasets.ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [11]:
model = vgg16_bn(pretrained=True)

In [14]:
# Freeze initial layers
for param in model.features.parameters():
    param.requires_grad = False

# Modify the last fully connected layer
model.classifier[6] = torch.nn.Linear(4096, 3)  # Replace num_classes with the number of classes in your dataset

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5  # Adjust this value based on your requirements
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}")


Epoch 1/5 - Loss: 1.1241
Epoch 2/5 - Loss: 0.9556
Epoch 3/5 - Loss: 0.7526
Epoch 4/5 - Loss: 0.6529
Epoch 5/5 - Loss: 0.5249


In [15]:
model.eval()
total_correct = 0
total_samples = 0

predicted_labels = []
actual_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        predicted_labels.extend(predicted.tolist())
        actual_labels.extend(labels.tolist())

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

accuracy = total_correct / total_samples
print(f"Accuracy on the test dataset: {accuracy*100:.2f}%")

# Print predicted and actual labels
print("Predicted labels: ", predicted_labels)
print("Actual labels: ", actual_labels)

Accuracy on the test dataset: 93.33%
Predicted labels:  [0, 0, 0, 0, 0, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2]
Actual labels:  [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
